# <center> Trabalho 00 - Introdução a Processamento de Imagens </center>

## Importar bibliotecas

In [1]:
import cv2 as cv
import numpy as np

## Questão 01 - Transformação de Intensidade

Para lermos uma imagem utilizando *opencv*, podemos utilizar a função *imread*, em que o primeiro argumento é o caminho para a imagem e o segundo argumento indica o espaço de cores no qual queremos carregá-la. No caso de uma imagem monocromática, podemos passar o valor 0 para o segundo argumento. Assim, a intensidade de um pixel será representada por apenas um valor.

A função *imread* retorna um array numpy, formando uma matriz em que cada linha corresponde a uma linha da imagem.

In [2]:
image = cv.imread('../03-input_images/city.png',0)

### a) Obter o negativo da imagem, ou seja, o nı́vel de cinza 0 será convertido para 255, o nı́vel 1 para 254 e assim por diante

Para conseguirmos obter o negativo de uma imagem, conforme especificado na questão, podemos usar a função *bitwise_not* da biblioteca *opencv*. Ela realiza a operação **not** em cada bit do cojunto de bits de cada pixel da imagem. 

Dessa forma, dada uma imagem em que cada pixel possui 8 bits, temos que os valores possíveis para a intensidade de cada pixel vão de 0 a 255. Assim, dado um pixel com valor 145, por exemplo, temos que sua representação binária é dada por: $10010001_2$. Ao aplicarmos a função *bitwise_not*, obteremos $01101110_2$, que equivale a $110_{10}$, que é o mesmo que 255-145 = 110.

In [3]:
new_image = cv.bitwise_not(image)

In [4]:
cv.imshow('negative',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q1_a_negative.png', new_image)

### b) Converter o intervalo de intensidades para [100, 200]

Para convertermos o intervalo de intensidades dos pixels, utilizamos uma função linear. Para construirmos essa função, adotamos o seguinte raciocínio: 

Quando o valor for 0, queremos que ele se torne 100, e quando ele for 255, queremos que ele se transforme em 200; entre 0 e 255 há 255 unidades, já entre 100 e 200 há 100 unidades; entretanto, para mapear o primeiro intervalo para o segundo, queremos que todos os valores que cabem no primeiro também tenham valores correspondentes no segundo; dessa forma, dividimos 100 por 255, ou seja, quebramos o comprimento do segundo intervalo em 255 unidades. Assim, o valor mínimo será 0 e o máximo será 100. Todavia, como queremos que eles estejam entre 100 e 200, somamos 100 ao resultado obtido. 

A função descrita pode ser observada na equação abaixo, em que x é o valor de intensidade de um pixel da imagem:

$$ f(x) = \frac{100}{255}*x + 100 $$

Um problema encontrado durante a resolução desse problema foi o tipo de dados da nova imagem gerada. Como estamos realizando uma operação de divisão em ponto flutuante, o resultado é um array de floats. Entretanto, a função *imshow* do *opencv* representa um array de floats como uma imagem em branco. Assim, convertemos os tipos dos elementos do array de saída para o tipo *uint8* com o método *astype*. Apesar disso, a função *imwrite* consegue salvar a imagem, mesmo que seus valores sejam do tipo float.

In [7]:
new_image = np.floor((100/255)*image + 100).astype(np.uint8)

In [8]:
cv.imshow('intensity',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q1_b_scala.png', new_image)

### c) Inverter os valores dos pixels das linhas pares da imagem, ou seja, os valores dos pixels da linha 0 serão posicionados da direita para esquerda, os valores dos pixels da linha 2 serão posicionados da direita para a esquerda e assim por diante

Neste questão, primeiros salvamos a imagem em uma estrutura auxiliar para não alterarmos a imagem original. Para selecionarmos apenas as linhas pares, percorremos o array acessando seus elementos a cada duas linhas, inclusive a primeira, como pode ser visto em <code>new_image[::2]</code>. 

Para invertermos os valores dos pixels, utilizamos o método *flip* do *opencv*. Como primeiro argumento, especificamos o array de valores que queremos inverter, e o segundo argumento indica em que direção inverteremos os valores (1 significa que giraremos em torno do eixo y).

In [9]:
new_image = image.copy()
new_image[::2] = cv.flip(new_image[::2],1)

In [10]:
cv.imshow('flip-odd',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q1_c_flip_odd.png', new_image)

### d) Espelhar as linhas da metade superior da imagem na parte inferior da imagem.

Para espelharmos as linhas da metade superior para a parte inferior, primeiro calculamos o meio da imagem, ou seja, a linha que se encontrava na metade da matriz. Depois percorremos a matriz a partir do meio para baixo, substituindo seus valores pelos valores da metade superior invertidos. Para invertermos a parte superior, aplicamos novamente o método *flip*, mas dessa vez passando o valor 0 como argumento, indicando que queríamos girar em torno do eixo x.

In [11]:
new_image = image.copy()
center = image.shape[0]//2
new_image[center:] = cv.flip(new_image[:center],0)

In [12]:
cv.imshow('mirror-top',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q1_d_mirror_top.png', new_image)

## Questão 2 - Ajuste de Brilho

In [13]:
image = cv.imread('../03-input_images/baboon.png', 0)

Para ajustarmos o brilho da imagem, utilizamos o passo a passo indicado no enunciado da questão. Primeiro, convertermos a intensidade dos pixels da imagem do intervalo de [0,255] para o de [0,1]. Então, aplicamos a equação $B = A^{\frac{1}{\gamma}}$, em que A é a imagem cuja escala foi transformada e B é a saída ao aplicarmos a transformação com Gama. Por fim, transformamos a escala de volta para o intervalo [0,255] e determinados que o tipo de dados de cada elemento do array que armazena a imagem de saída deve ser do tipo *uint8*.

In [14]:
A = image/255
gamma = 3.5
new_image = (A**(1/gamma)*255).astype(np.uint8)

In [15]:
cv.imshow('brightness',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q2_brightness35.png', new_image)

## Questão 03 - Plano de Bits

In [16]:
image = cv.imread('../03-input_images/baboon.png', 0)

Planos de bits consistem em uma representação dos valores de cada bit dos pixels de uma imagem. Em outras palavras, imagine uma imagem de 8 bits, ou seja, cada um de seus pixels possui 8 bits - valores podem ir de 0 a 255. Essa imagem possui 8 planos de bits, que vão desde o plano 0 até o plano 7. No plano 0 representamos os valores do bit 0 de cada pixel, no bit 1 representamos os valores do bit 1 de cada pixel e assim por diante. Dessa maneira, cada plano corresponde a uma matriz com as mesmas dimensões da imagem original, mas com os valores dos bits correspondentes ao plano observado.

Para calcularmos os planos de bits da imagem monocromática carregada, fazemos uma operação *bitwise and* para o bit queremos. Para isso, elevamos 2 à posição do bit, o que resultará em um valor cuja representação binária possuirá 1 apenas na posição do bit desejado. Então, deslocamos o valor obtido da operação bitwise para a direita, até que ele chegue à posição 0. Isso resultará em valores 0 ou 1. Por fim, como queremos que 0 seja o valor mínimo e 1 seja o valor máximo, multiplicamos o resultado por 255. Um exemplo pode ser visto abaixo:

Imagine que queremos o valor do bit 4 do pixel cujo valor é $10010101_2$, então fazemos $2^4 = 16_{10} = 00010000_2$. A seguir, realizamos a operação *bitwise and*: 10010101 & 00010000 = 00010000. Então, deslocamos o valor para a direita 4 posições: (1)0000**1**000 -> (2)00000**1**00 -> (3)000000**1**0 -> (4)0000000**1**. Assim, descobrimos que o valor do bit 4 do pixel observado é 1. Multiplicar por 255 fará com que 0 produza o pixel mais escuro, enquanto 1 produzirá o pixel mais claro.

In [17]:
n_bits = 8
planes = {}

for k in range(0,n_bits):
    planes[k] = ((image & 2**k) >> k)*255

In [18]:
n = 7
cv.imshow('bit-plane',planes[n])
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q3_bit_plane'+str(n)+'.png', planes[n])

## Questão 04 - Mosaico

In [19]:
image = cv.imread('../03-input_images/baboon.png', 0)

Para resolver essa questão, primeiro determinamos o número de linhas (n_lines) e colunas (n_cols) da imagem, em quantas partes ela seria divida horizontal e verticalmente (n_slices) e a quantidade de pixels por linha (n_cols_slice) e por coluna (n_lines_slice) de cada parte. Também, criamos uma lista (pattern) com a ordem em que cada parte da imagem deveria ser reorganizada. Também foram criados um dicionário, para armazenar as partes a serem retiradas da imagem, e uma matriz (mosaic) com a mesma dimensão da imagem original, para armazenar o mosaico.

O próximo passo após definir as estruturas foi percorrer a imagem original, retirando pedaços de tamanho n_cols_slice * n_lines_slice e salvando na estrutura *img_slices* (primeiro par de loops). A imagem foi percorrida da esquerda para a direita e de cima para baixo.

Uma vez possuindo todos os pedaços da imagem original, preencheu-se a matriz *mosaic*, também da esquerda para a direita e de cima para baixo, mas colocando a parte correspondente à ordem estabelecida na estrutura *pattern*.

In [20]:
n_cols,n_lines = image.shape
n_slices = 4
n_cols_slice = n_cols//4 
n_lines_slice = n_lines//4
pattern = [6,11,13,3,8,16,1,9,12,14,2,7,4,15,10,5]

img_slices = {}
mosaic = np.zeros(shape=(n_cols, n_lines),dtype=np.uint8)

In [21]:
pos = 0
for col_pos in range(0,n_slices):
    for line_pos in range(0,n_slices):
        img_slices[pos] = image[col_pos*n_cols_slice:n_cols_slice*(1 + col_pos), \
                                line_pos*n_lines_slice:n_lines_slice*(1 + line_pos)]
        pos = pos+1

pos = 0
for col_pos in range(0,n_slices):
    for line_pos in range(0,n_slices):
        mosaic[col_pos*n_cols_slice:n_cols_slice*(1 + col_pos), \
            line_pos*n_lines_slice:n_lines_slice*(1 + line_pos)] = img_slices[pattern[pos]-1]
        pos = pos + 1

In [22]:
cv.imshow('mosaic',mosaic)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q4_mosaic.png', mosaic)

## Questão 05 - Combinação de Imagens

In [23]:
image1 = cv.imread("../03-input_images/baboon.png",0)
image2 = cv.imread("../03-input_images/butterfly.png",0)

Para resolver essa questão, usamos o método *addWeighted* do *opencv*. Ele realiza a soma ponderada das imagens, ou seja, multiplica cada imagem pelo respectivo coeficiente antes de somá-las. O primeiro argumento é a estrutura que representa a primeira imagem, o segundo é o coeficiente que multiplicará seus elementos, o terceiro argumento é a segunda imagem, o quarto é o coeficiente que multiplicará os elementos desta e o quinto argumento, chamado de *bias*, adiciona um valor ao resultado da operação. A Equação abaixo apresenta a fórmula calculada por esse método.

$$dst = src1*alpha + src2*beta + gamma$$

In [24]:
new_image = cv.addWeighted(image1,0.8,image2,0.2,0)

In [25]:
cv.imshow('image-blending',new_image)
cv.waitKey(0)
cv.destroyAllWindows()
#cv.imwrite('result-images/q5_blending82.png', new_image)